In [1]:
from amplpy import AMPL, ampl_notebook
import pandas
import matplotlib.pyplot as plt
import numpy

In [2]:
license = "a08a4945-4807-4400-83da-e35bdca430e4"
ampl = ampl_notebook(
    modules=["highs", "cbc", "gurobi", "cplex"], # pick from over 20 modules including most commercial and open-source solvers
    license_uuid=license) # La licencia del curso. Si la cambia, esto no funciona.

Licensed to AMPL Academic Community Edition License for <j.palmaespinosa@uandresbello.edu>.



# Introducción

En los talleres anteriores Ud. ha trabajado con códigos AMPL y ha resuelto un conjunto acotado de problemas.  Como recordará, la potencia de AMPL radica en poder escribir el modelo de forma general, de forma que Ud. se concentre en modelar, no en resolver.  
En este taller, el objetivo será justo ese: Modelar.

Para ello, se incluyeron un conjunto de datos para diferentes problemas (camino mínimo, flujo máximo y flujo a costo mínimo).
Ud. deberá escribir el modelo general en AMPL y resolver cada uno de los problemas, cargando los datos correspondientes.

**Nota: observe que los datos están en la carpeta _data_** 

---

## Problema 1
Considere el problema general de camino mínimo.  



1. Escriba el modelo general, en papel.
2. implemente dicho modelo en AMPL.
3. Revise los datos correspondientes y dibuje la red.
4. Corra el modelo usando el conjunto de datos correspondientes y **verifique que el tiempo total empleado es 18**
5. Dibuje la ruta recorrida.

---

In [4]:
#Recuerde %%writefile caminoMinimo.mod
# Codigo AMPL

Writing caminoMinimo.mod


In [9]:
# 1. Limpiamos la sesion anterior y creamos una nueva
ampl.close()
ampl = AMPL()
# 2. Cargar el modelo dado un archivo MODELO.mod
ampl.read("caminoMinimo.mod")
# 3. CARGAR LOS DATOS DEL PROBLEMA
ampl.read_data("data_camino.dat")
# 3. Resolver con algún solver (puede revisar la documentación de AMPL).
ampl.solve(solver="cplex")

assert ampl.solve_result == "solved"

# Mostramos la respuesta.
for var in ["mi_variable"]:
    ampl.display(var)

CPLEX 22.1.2: optimal solution; objective 18
0 simplex iterations
Use :=
a b   0
a c   1
a d   0
b f   0
b g   0
c b   0
c e   0
c f   1
d c   0
d e   0
e f   0
e z   0
f g   0
f z   1
g z   0
;



---
## Problema 2
Considere el problema general de Flujo máximo.  


1. Escriba el modelo general, en papel.
2. Implemente dicho modelo en AMPL.
3. Revise los datos correspondientes y dibuje la red.
4. Corra el modelo usando el conjunto de datos correspondientes y **verifique que el flujo máximo es 35**
5. Dibuje el resultado obtenido.

In [14]:
#%%writefile maxFlow.mod


In [13]:
# 1. Limpiamos la sesion anterior y creamos una nueva
ampl.close()
ampl = AMPL()
# 2. Cargar el modelo dado un archivo MODELO.mod
ampl.read("maxFlow.mod")
# 3. CARGAR LOS DATOS DEL PROBLEMA
ampl.read_data("data.dat")
# 3. Resolver con algún solver (puede revisar la documentación de AMPL).
ampl.solve(solver="cplex")

assert ampl.solve_result == "solved"

# Mostramos la respuesta.
for var in ["mi_variable"]:
    ampl.display(var)

CPLEX 22.1.2: optimal solution; objective 35
0 simplex iterations
Entering_Traff = 35



---
## Problema 3
Considere el problema general de Flujo a costo mínimo.  

1. Escriba el modelo general, en papel.
2. Implemente dicho modelo en AMPL.
3. Revise los datos correspondientes y dibuje la red.
4. Corra el modelo usando el conjunto de datos correspondientes y **verifique que el costo mínimo es 870 y que el flujo enviado es de 470**
5. Dibuje el resultado obtenido.

1. Escriba el modelo general, en papel
2. implemente dicho modelo en AMPL.
3. Corra el modelo usando el conjunto de datos correspondientes y 

In [15]:
%%writefile minFlow.mod

set NODES;
set LINKS within(NODES cross NODES);  # edges between each node in the system

param supply{NODES}>=0;  # Nodes
param demand{NODES}>=0;
param cost{LINKS} >= 0;   # shipment costs per unit
param capacity{LINKS}>=0; # Capacity for source/sink

var X{(i,j) in LINKS} >= 0, <= capacity[i,j];

minimize Total_Cost:
   sum {(i,j) in LINKS} cost[i,j] * X[i,j];

subject to Balance {k in NODES}:
      sum {(i,k) in LINKS} X[i,k] - sum {(k,j) in LINKS}X[k,j]
        = demand[k] - supply[k] ;


Writing minFlow.mod


In [20]:
# 1. Limpiamos la sesion anterior y creamos una nueva
ampl.close()
ampl = AMPL()
# 2. Cargar el modelo dado un archivo MODELO.mod
ampl.read("minFlow.mod")
# 3. CARGAR LOS DATOS DEL PROBLEMA
ampl.read_data("./data/minFlow_data.dat")
# 3. Resolver con algún solver (puede revisar la documentación de AMPL).
ampl.solve(solver="cplex")

assert ampl.solve_result == "solved"

# Mostramos la respuesta.
for var in ["X"]:
    ampl.display(var)

CPLEX 22.1.2: optimal solution; objective 870
1 simplex iterations
X :=
D1 D3   120
D2 D3     0
S1 D1   100
S1 S2     0
S1 S3     0
S2 D1   170
S2 D2    30
S3 D2    50
S3 S2     0
;



Cómo pudo observar, la idea es simple:

```python
# 1. Cargar AMPL
ampl = AMPL()
# 2. Cargar el modelo dado un archivo MODELO.mod
ampl.read("MODELO.mod")
# 3. Resolver con algún solver (puede revisar la documentación de AMPL).
ampl.solve(solver="cplex")
```

Use la misma estructura para resolver el siguiente problema:


> Una empresa fabrica tres productos: A, B y C. Cada uno requiere tiempo de máquina, material y mano de obra, con las siguientes necesidades por unidad producida:


| Recurso           | Producto A | Producto B | Producto C | Disponibilidad |
|:-----------------:|:----------:|:----------:|:----------:|:--------------|
| Tiempo (h)        |          2 |          1 |          3 |            100 |
| Material (kg)     |          3 |          4 |          2 |            120 |
| Mano de obra (h)  |          1 |          2 |          1 |             90 |


> Además, cada producto genera una utilidad:
> - A: 30 unidades monetarias
> - B: 20 unidades monetarias
> - C: 40 unidades monetarias
>   
> Determine cuántas unidades de cada producto deben producirse para maximizar la utilidad total, respetando las restricciones de recursos.

In [ ]:
# RESOLUCION EJEMPLO 2

---